# SAP Data Ingestion Automation - Notebook Mode

**For most team members:** Use `python main.py` in terminal instead!

See README.md for quick start guide.

This notebook is for development/debugging only.

In [ ]:
import os
import sys
import pandas as pd

src_path = os.path.abspath("../src")
sys.path.insert(0, src_path)

%load_ext autoreload
%autoreload 2

from utils import *
from dag_creation import *
from bucket import *

,icdsTableName,BANNER_NAME,dataSensitivity,OP-Company code,dlSchemaName,dlTableName,tableLoadType,keyPreCombine,keyPrimaryKey,bucket_id
0,IKPF,"MDD,MAK,MSB",se,"SA-MDD,SA-MAK,SA-MSB",sa_mdse_dl_secure,PHYSL_INVT_DOC,INC,ds_load_ts,"clnt,application,cond_type,sales_org,distr_cha...",123
1,PF,"MDD,MAK,MSB",se,"SA-MDD,SA-MAK,SA-MSB",sa_mdse_dl_table,PHYSL_INVT_DOC,INC,ds_load_ts,"clnt,application,cond_type,sales_org,distr_cha...",345


## Advanced: Manual DAG Preparation

Use this section only if you need to customize the pipeline.
For standard ingestions, use: `python main.py --env dev`

In [ ]:
filename = "../ingestion.csv"
df = pd.read_csv(filename)
print(f"Loaded {len(df)} tables")
df.head()

In [ ]:
# ADVANCED USERS ONLY: Manual execution
extract_bucket_id()

def prepare_dag_configuration(df):
    # iterate for each row of dataframe
    
    dag_list = [] # it will collect all the dags created 
    for index, row in df.iterrows():
        tableLoadType = row['tableLoadType']  # INC
        dlSchemaName = row['dlSchemaName']  # example : sa_mdse_dl_secure
        dlTableName = row['dlTableName']  # example : PHYSL_INVT_DOC
        dataSensitivity = row['dataSensitivity']  # SE/NS/HS
        banner_list = row['BANNER_NAME'].split(',')  # [MDD,MAK,MSB]
        
        table_names = prepare_table_name(banner_list, dlTableName)
        
        cluster_names = prepare_cluster_name(dlSchemaName, banner_list, dlTableName)
        
        sample_dag_file = "../sample_dag.py"
        sample_sql_file = "../sample_sql.sql"
        
        # for each banner, there will be dag_config
        for i, b in enumerate(banner_list):
            dag_config = {
                "sensitivity": dataSensitivity,
                "cluster_name": cluster_names[i],
                "banner_name": b,
                "table_name": table_names[i],
                "tags": ["Massmart-eComm","P2","Ephemeral","SA","SECURE","MDSE",f"{b}",f"{table_names[i]}","SLT"],
                "tableLoadType": tableLoadType,
                "output_dir": f"../output/{dlSchemaName}/{table_names[i]}",
                "dag_name": f"INTLDLDAT-SA{b}-{tableLoadType}-{dlSchemaName.upper()}-{b}_{dlTableName}",
            }
            # for each dag , generate input to create a bcuket and append in csv file
            create_bucket(dag_config, dlSchemaName, output_file="../buckets/bucket_input.csv", env='prod')
            
            bucket_id = get_bucket_id(dag_config)
            
            prepare_dag_file(sample_dag_file, dag_config)
            
            prepare_sql_file(sample_sql_file, dag_config, dlSchemaName, dlTableName, bucket_id)
            
            dag_list.append(dag_config)
    
    return dag_list

✓ Successfully created ../buckets/bucket_id.csv
✓ Extracted 10 table-bucket mappings

Sample mappings:
                  dlTableName  \
0          mdd_physl_invt_doc   
1          mak_physl_invt_doc   
2          msb_physl_invt_doc   
3  mdd_article_promo_type_txt   
4  mak_article_promo_type_txt   

                                           bucket_id  
0  676e39d3ff0cc7e5790e2d370b147db52079cf1a6dbf57...  
1  86009702bc2c3bcd2c1c64cfb8e9a2e7ed7046d5e4b777...  
2  b9d5873094694ab819ba71c0018625cd49ce86fd42f0bc...  
3  21dcaf6ee5b8edc50e7916119c895fdf08776a2cf40dd7...  
4  2ba2595ae3fa67e996046b483952b39dd0874d77382907...  


In [ ]:
dag_list = prepare_dag_configuration(df)
print(f"Generated {len(dag_list)} DAGs")

Creating bulk bucket ... 
✓ Added bucket entry for mdd_physl_invt_doc to ../buckets/bucket_input.csv
✓ Retrieved bucket_id for mdd_physl_invt_doc: 676e39d3ff0cc7e5790e2d370b147db52079cf1a6dbf57ff0f33d652ae40be
✓ Created SQL file: ../output/sa_mdse_dl_secure/mdd_physl_invt_doc/mdd_physl_invt_doc.sql
Creating bulk bucket ... 
✓ Added bucket entry for mak_physl_invt_doc to ../buckets/bucket_input.csv
✓ Retrieved bucket_id for mak_physl_invt_doc: 86009702bc2c3bcd2c1c64cfb8e9a2e7ed7046d5e4b77786956ff1fe50586d
✓ Created SQL file: ../output/sa_mdse_dl_secure/mak_physl_invt_doc/mak_physl_invt_doc.sql
Creating bulk bucket ... 
✓ Added bucket entry for msb_physl_invt_doc to ../buckets/bucket_input.csv
✓ Retrieved bucket_id for msb_physl_invt_doc: b9d5873094694ab819ba71c0018625cd49ce86fd42f0bcb15c76599bf3f872
✓ Created SQL file: ../output/sa_mdse_dl_secure/msb_physl_invt_doc/msb_physl_invt_doc.sql
Creating bulk bucket ... 
✓ Added bucket entry for mdd_physl_invt_doc to ../buckets/bucket_input.csv

## Upload to Airflow

In [ ]:
upload_commands = generate_upload_commands(
    source_dir="../output",
    dest_bucket="gs://bfdaf-dags-intldlsadev-catalog/",
    output_file="../upload_commands.sh"
)

print(f"\nGenerated {len(upload_commands)} upload commands")
print("Run: bash ../upload_commands.sh")

Found 6 DAG files
gcloud storage cp ../output/sa_mdse_dl_secure/mak_physl_invt_doc/INTLDLDAT-SAMAK-INC-SA_MDSE_DL_SECURE-MAK_PHYSL_INVT_DOC.py gs://bfdaf-dags-intldlsadev-catalog/
gcloud storage cp ../output/sa_mdse_dl_secure/mdd_physl_invt_doc/INTLDLDAT-SAMDD-INC-SA_MDSE_DL_SECURE-MDD_PHYSL_INVT_DOC.py gs://bfdaf-dags-intldlsadev-catalog/
gcloud storage cp ../output/sa_mdse_dl_secure/msb_physl_invt_doc/INTLDLDAT-SAMSB-INC-SA_MDSE_DL_SECURE-MSB_PHYSL_INVT_DOC.py gs://bfdaf-dags-intldlsadev-catalog/
gcloud storage cp ../output/sa_mdse_dl_table/mak_physl_invt_doc/INTLDLDAT-SAMAK-INC-SA_MDSE_DL_TABLE-MAK_PHYSL_INVT_DOC.py gs://bfdaf-dags-intldlsadev-catalog/
gcloud storage cp ../output/sa_mdse_dl_table/mdd_physl_invt_doc/INTLDLDAT-SAMDD-INC-SA_MDSE_DL_TABLE-MDD_PHYSL_INVT_DOC.py gs://bfdaf-dags-intldlsadev-catalog/
gcloud storage cp ../output/sa_mdse_dl_table/msb_physl_invt_doc/INTLDLDAT-SAMSB-INC-SA_MDSE_DL_TABLE-MSB_PHYSL_INVT_DOC.py gs://bfdaf-dags-intldlsadev-catalog/

✓ Commands save